# Flask-SQLAlchemy

Flask itself does not come with any database backend, but the Flask-SQLAlchemy plugin provides database access via the SQLAlchemy database toolkit.



In [1]:
!pip install flask-sqlalchemy

You are using pip version 18.1, however version 19.0.3 is available.
You should consider upgrading via the 'pip install --upgrade pip' command.


In [2]:
from flask_helpers import running_app

In [3]:
%%file data/flask-examples/sa_model.py
from flask_sqlalchemy import SQLAlchemy

db = SQLAlchemy()

class Post(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    postedDate = db.Column(db.DateTime)
    authorName = db.Column(db.String)
    title = db.Column(db.String)
    content = db.Column(db.String)
    
class Comment(db.Model):
    id = db.Column(db.Integer, primary_key=True)
    post_id = db.Column(db.Integer, db.ForeignKey('post.id'), nullable=False)
    postedDate = db.Column(db.DateTime)
    authorName = db.Column(db.String)
    content = db.Column(db.String)
    
    post = db.relationship('Post', backref=db.backref('comments', lazy=True))


Overwriting data/flask-examples/sa_model.py


In [4]:
%%file data/flask-examples/sa-app.py
from datetime import datetime

from flask import Flask, url_for, jsonify, request, abort

from sa_model import db, Post, Comment

app = Flask(__name__)
app.config['SQLALCHEMY_DATABASE_URI'] = 'sqlite:///blog.db'
app.config['SQLALCHEMY_TRACK_MODIFICATIONS'] = False
db.init_app(app)


@app.route('/')
def get_root():
    return jsonify(_links={'posts': url_for('get_posts', _external=True)})


@app.route('/post')
def get_posts():
    post_links = [
        url_for('get_post', post_id=post.id, _external=True) 
        for post in Post.query]
    return jsonify(
        _links={'self': url_for('get_posts', _external=True)},
        posts=[dict(_links=dict(self=link)) for link in post_links])

@app.route('/post', methods=['POST'])
def create_post():
    post = Post(
        authorName=request.authorization.username,
        postedDate=datetime.utcnow(),
        title=request.json['title'],
        content=request.json['content'],
    )
    db.session.add(post)
    db.session.commit()
    result = jsonify_post(post)
    result.headers['Location'] = url_for('get_post', post_id=post.id, _external=True)
    return result

@app.route('/post/<int:post_id>')
def get_post(post_id):
    post = Post.query.get(post_id)
    if not post:
        abort(404)
    return jsonify_post(post)

@app.route('/post/<int:post_id>', methods=['PUT'])
def update_post(post_id):
    post = Post.query.get(post_id)
    if not post:
        abort(404)
    post.authorName = request.authorization.username
    post.postedDate = datetime.utcnow()
    post.title = request.json['title']
    post.content = request.json['content']
    db.session.commit()
    return jsonify_post(post)

@app.route('/post/<int:post_id>', methods=['DELETE'])
def delete_post(post_id):
    post = Post.query.get(post_id)
    if not post:
        abort(404)
    db.session.delete(post)
    db.session.commit()
    return '', 204

def jsonify_post(post):
    return jsonify(
        _links={'self': url_for('get_post', post_id=post.id, _external=True)},
        postedDate=post.postedDate.isoformat(),
        authorName=post.authorName,
        title=post.title,
        content=post.content,
    )

Overwriting data/flask-examples/sa-app.py


In [5]:
import requests
sess = requests.Session()
sess.auth = ('rick', 'password')

In [8]:
with running_app('data/flask-examples/sa-app.py'):
    sess.post('http://localhost:5000/post', json={
        'title': 'First post!',
        'content': 'This is the first post, does it work?',
    })

 * Serving Flask app "data/flask-examples/sa-app.py"
 * Environment: development
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Mar/2019 11:41:17] "POST /post HTTP/1.1" 200 -
Exiting output thread


In [10]:
import sqlite3
conn = sqlite3.connect('data/flask-examples/blog.db')

In [12]:
for row in conn.execute('SELECT * FROM post'):
    print(row)

(1, '2019-03-22 18:41:17.540083', 'rick', 'First post!', 'This is the first post, does it work?')


In [7]:
with running_app('data/flask-examples/sa-app.py'):
    resp = sess.get('http://localhost:5000/post')
    resp.raise_for_status()
    print(resp.json())
    for post in resp.json()['posts']:
        print('Post')
        r = sess.get(post['_links']['self'])
        r.raise_for_status()
        print(r.json())
        sess.delete(post['_links']['self'])

 * Serving Flask app "data/flask-examples/sa-app.py"
 * Environment: development
 * Debug mode: on
 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [22/Mar/2019 11:41:02] "GET /post HTTP/1.1" 200 -
{'_links': {'self': 'http://localhost:5000/post'}, 'posts': [{'_links': {'self': 'http://localhost:5000/post/1'}}]}
Post
127.0.0.1 - - [22/Mar/2019 11:41:02] "GET /post/1 HTTP/1.1" 200 -
{'_links': {'self': 'http://localhost:5000/post/1'}, 'authorName': 'rick', 'content': 'This is the first post, does it work?', 'postedDate': '2019-03-22T18:40:52.912340', 'title': 'First post!'}
127.0.0.1 - - [22/Mar/2019 11:41:02] "DELETE /post/1 HTTP/1.1" 204 -
Exiting output thread


Open the [Flask-SQLAlchemy lab](./flask-sqlalchemy-lab.ipynb)